# Data Acquisition 

The very first step of any computational experiment is always obtaining data. Most of the times this is the very tedious part and it pays off to spend a lot of time understanding, cleaning and refining your data set. For the sake of simplicity, we will take a few shortcuts in this tutorial, but we will try to hint at possibilities to improve the process as we go on.

## Papyrus Data Set

The [Papyrus](https://chemrxiv.org/engage/chemrxiv/article-details/617aa2467a002162403d71f0) is a curated data set that tries to make our lives easier by agregating publically available data under one roof and already does a few things for us, such as standardizing the compounds, aggregating and transforming activity values and giving our data overall structure that is suitable for data analysis and machine learning. However, there are also downsides to using such data set. For example, the fact that this is a curated data set means that it needs to be updated each time new information becomes available so you might not always have the most recent data to work with. In addition, sometimes the form of standardization of the data set might not be suitable for your case so you still need to spend some time reading about how the data is curated and if valuable information is not lost. For example, Papyrus does not include stereochemistry by default so if you want that kind of data (i.e. for docking), you have to retrieve it seperately.

In this tutorial, we will be using a tool (`papyrus-scaffold-visualizer`) that enables easy extraction of data from Papyrus, but also has some simple chemical space depiction features. First, we need UniProt accession keys of our protein of interest. For the purpose of this tutorial, it will be the [Adenosine A2A receptor](https://www.uniprot.org/uniprotkb/P29274/entry):

In [2]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# define work directory to store data
DATA_ROOT = '/content/drive/MyDrive/DrugExDemo/' # or wherever you want the generated files to live on your GoogleDrive
import os
os.makedirs(DATA_ROOT, exist_ok=True)
os.chdir(DATA_ROOT) 

# fetch pretrained model
os.makedirs("./data/drugex/models/pretrained/", exist_ok=True)
! wget -nc -P './data/drugex/models/pretrained/' 'https://zenodo.org/record/7096859/files/DrugEx_v2_PT_Papyrus05.5.zip'
! unzip -n './data/drugex/models/pretrained/DrugEx_v2_PT_Papyrus05.5.zip' -d './data/drugex/models/pretrained/DrugEx_v2_PT_Papyrus05.5'

# install dependencies
! git clone https://github.com/martin-sicho/drugex-demo
! pip install -r drugex-demo/requirements.txt

# verify where we are working
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File ‘./data/drugex/models/pretrained/DrugEx_v2_PT_Papyrus05.5.zip’ already there; not retrieving.

Archive:  ./data/drugex/models/pretrained/DrugEx_v2_PT_Papyrus05.5.zip
fatal: destination path 'drugex-demo' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/martin-sicho/DrugEx-CDDG.git (to revision master) to /tmp/pip-install-zbvevyps/drugex_a10de01965014311bce80b641e9bf436
  Running command git clone -q https://github.com/martin-sicho/DrugEx-CDDG.git /tmp/pip-install-zbvevyps/drugex_a10de01965014311bce80b641e9bf436
  Cloning https://github.com/martin-sicho/papyrus-scaffold-visualizer.git (to revision v0.2.0) to /tmp/pip-install-zbvevyps/papyrus-scaffold-visualizer_04eb96643a704b0ca20bad06f1426025
  Running command git clone -q https://

'/content/drive/MyDrive/DrugExDemo'

In [3]:
!pip install git+https://github.com/martin-sicho/DrugEx-CDDG.git@master
!pip install git+https://github.com/martin-sicho/papyrus-scaffold-visualizer.git@v0.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/martin-sicho/DrugEx-CDDG.git (to revision master) to /tmp/pip-req-build-10sapv1n
  Running command git clone -q https://github.com/martin-sicho/DrugEx-CDDG.git /tmp/pip-req-build-10sapv1n
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/martin-sicho/papyrus-scaffold-visualizer.git (to revision v0.2.0) to /tmp/pip-req-build-1h0qkc34
  Running command git clone -q https://github.com/martin-sicho/papyrus-scaffold-visualizer.git /tmp/pip-req-build-1h0qkc34
  Running command git checkout -q ff4f2e885a3973f90a0d9864dfa00abed493f78d
  Cloning https://github.com/OlivierBeq/Papyrus-scripts.git (to revision master) to /tmp/pip-install-7rj0bbwu/papyrus-scripts_80bb0def3e3e48f3a74c3169522551fc
  Running command git clone -q https://github.com/OlivierBeq/Papyrus-scripts.git /tmp/pip-install-7rj0

In [4]:
acc_keys = ["P18031"] # Adenosine receptor PTP1B (https://www.uniprot.org/uniprotkb/P29274/entry)
name = "PTP1B_LIGANDS" # name of the file to be generated
quality = "medium" # choose minimum quality from {"high", "medium", "low"}

These will be our basic settings for the data extraction from Papyrus. In short, we will get compound structures and their measured activities on the A2A receptor that should be of `low` quality or higher as specified by Papyrus' data curation workflow. For the purpose of this tutorial, we are interested in all available data (`low` and higher quality), but in some other applications you might want a more stringent filter. We suggest you read the documentation of Papyrus to learn how these quality classes are determined.

The `Papyrus` class exposes some basic configuration features with the help of the more powerful [`papyrus_scripts`](https://github.com/OlivierBeq/Papyrus-scripts) package. It will fetch the latest available version of Papyrus data automatically for us as as specified. Here, we are asking only for the latest Papyrus data files without definitions of stereochemistry and all data will be placed in the `./data` directory:

In [5]:
from scaffviz.data.papyrus import Papyrus

# fetches latest version of Papyrus if not already available and filters out A2A data for us
papyrus = Papyrus(
    data_dir="./data", # data storage directory 
    stereo=False, # do not include stereochemistry
    version='latest'
)

No data was transferred yet, but once we attempt to filter for our data, the data should start downloading:

In [6]:
dataset = papyrus.getData(
    acc_keys,
    quality,
    name,
    use_existing=True # use existing data set if it was already compiled before to speed things up
)

Latest version: 05.5
Number of files to be donwloaded: 6
Total size: 118MB


Donwloading version 05.5:   0%|          | 0.00/118M [00:00<?, ?B/s]

Using existing data from ./data/PTP1B_LIGANDS.tsv...


The Papyrus data will only be fetched once if the latest version is already available in the `./data` folder. Plus, thanks to the `use_existing` parameter the created data set file will also be reused once extracted. You can find it here:

In [7]:
dataset.path

'./data/PTP1B_LIGANDS.tsv'

It is no special file and you can either use `pandas` to load it manually into a `DataFrame` in your environment or get it sraight from the instance returned by the `getData()` method:

In [8]:
dataset.asDataFrame().head()

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,Descriptor_MorganFP_1016,Descriptor_MorganFP_1017,Descriptor_MorganFP_1018,Descriptor_MorganFP_1019,Descriptor_MorganFP_1020,Descriptor_MorganFP_1021,Descriptor_MorganFP_1022,Descriptor_MorganFP_1023,TSNE_1,TSNE_2
0,AADVGRCQGZZZNH_on_P18031_WT,High,ChEMBL30,CHEMBL363338;44397840;CHEMBL363338;44397840;CH...,CC(C)CC(CC(=O)C(Cc1ccc(OC(F)(F)C(=O)O)cc1)NC(=...,AADVGRCQGZZZNH,AADVGRCQGZZZNH-UHFFFAOYSA-N,InChI=1S/C45H47F2N3O10/c1-27(2)22-30(41(48)53)...,"""AuxInfo=1/1/N:1,3,37,36,38,49,56,48,55,35,39,...",P18031_WT,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,29.735216,-4.313994
1,AAGZFGSRAZMRCS_on_P18031_WT,High,ChEMBL30,CHEMBL511000;CHEMBL511000;44563281;CHEMBL51100...,CC1(C)CCC2(C(=O)Nc3ccc(C(=O)O)cc3)CCC3(C)C(=CC...,AAGZFGSRAZMRCS,AAGZFGSRAZMRCS-UHFFFAOYSA-N,InChI=1S/C37H53NO4/c1-32(2)18-20-37(31(42)38-2...,"""AuxInfo=1/1/N:1,3,34,35,28,22,40,12,17,11,18,...",P18031_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.945723,55.179680
2,AATACZKOPAGNPL_on_P18031_WT,High,ChEMBL30,CHEMBL4160013,CCOc1ccc(OC(=O)CSc2nnc(-c3cc(O)c(O)cc3)o2)cc1,AATACZKOPAGNPL,AATACZKOPAGNPL-UHFFFAOYSA-N,InChI=1S/C18H16N2O6S/c1-2-24-12-4-6-13(7-5-12)...,"""AuxInfo=1/0/N:1,2,24,5,27,6,26,23,18,11,17,4,...",P18031_WT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.630247,-24.217190
3,AAWBMDOJDQKGGQ_on_P18031_WT,High,ChEMBL30,CHEMBL3759405,CCCOc1ccc(-c2cccc(-c3noc(Cc4c[nH]c5ccccc45)n3)...,AAWBMDOJDQKGGQ,AAWBMDOJDQKGGQ-UHFFFAOYSA-N,InChI=1S/C25H22N4O2/c1-2-14-30-19-12-10-17(11-...,"""AuxInfo=1/0/N:1,2,25,24,11,26,23,10,12,7,30,6...",P18031_WT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.455896,-43.069510
4,ABDQNRCWGSGNBQ_on_P18031_WT,High,ChEMBL30,CHEMBL197929,O=C(O)COc1ccc(S(=O)(=O)N(Cc2ccc(C(F)(F)P(=O)(O...,ABDQNRCWGSGNBQ,ABDQNRCWGSGNBQ-UHFFFAOYSA-N,"InChI=1S/C21H20F2NO8PS2/c22-21(23,33(27,28)29)...","""AuxInfo=1/1/N:31,30,16,27,17,26,7,35,8,34,32,...",P18031_WT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.058834,5.535724


This is a table you can already work with, but the `dataset` object is an instance of the `scaffviz.data.dataset.DataSetTSV` class that supports some more cheminformatics features that might be of interest: 

In [9]:
type(dataset)

scaffviz.data.dataset.DataSetTSV

For example, we can easily attach predefined or our own molecular descriptors if we need to (however, note that `Papyrus` class itself also allows you to define descriptors to fetch with the data):

In [10]:
from scaffviz.clustering.descriptors import MorganFP

# add Morgan fingerprints to the data set -> will be used to calculate the t-SNE embedding for chemical space visualization
dataset.addDescriptors([MorganFP(radius=2, nBits=1024)], recalculate=False) 
dataset.getDescriptors()

,Descriptor_MorganFP_0,Descriptor_MorganFP_1,Descriptor_MorganFP_2,Descriptor_MorganFP_3,Descriptor_MorganFP_4,Descriptor_MorganFP_5,Descriptor_MorganFP_6,Descriptor_MorganFP_7,Descriptor_MorganFP_8,Descriptor_MorganFP_9,...,Descriptor_MorganFP_1014,Descriptor_MorganFP_1015,Descriptor_MorganFP_1016,Descriptor_MorganFP_1017,Descriptor_MorganFP_1018,Descriptor_MorganFP_1019,Descriptor_MorganFP_1020,Descriptor_MorganFP_1021,Descriptor_MorganFP_1022,Descriptor_MorganFP_1023
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1768,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1769,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


If we ever want to use this class again to modify our data, we can recreate it like so:

In [11]:
from scaffviz.data.dataset import DataSetTSV

DataSetTSV('data/PTP1B_LIGANDS.tsv').asDataFrame().head()

,Activity_ID,Quality,source,CID,SMILES,connectivity,InChIKey,InChI,InChI_AuxInfo,target_id,...,Descriptor_MorganFP_1016,Descriptor_MorganFP_1017,Descriptor_MorganFP_1018,Descriptor_MorganFP_1019,Descriptor_MorganFP_1020,Descriptor_MorganFP_1021,Descriptor_MorganFP_1022,Descriptor_MorganFP_1023,TSNE_1,TSNE_2
0,AADVGRCQGZZZNH_on_P18031_WT,High,ChEMBL30,CHEMBL363338;44397840;CHEMBL363338;44397840;CH...,CC(C)CC(CC(=O)C(Cc1ccc(OC(F)(F)C(=O)O)cc1)NC(=...,AADVGRCQGZZZNH,AADVGRCQGZZZNH-UHFFFAOYSA-N,InChI=1S/C45H47F2N3O10/c1-27(2)22-30(41(48)53)...,"""AuxInfo=1/1/N:1,3,37,36,38,49,56,48,55,35,39,...",P18031_WT,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,29.735216,-4.313994
1,AAGZFGSRAZMRCS_on_P18031_WT,High,ChEMBL30,CHEMBL511000;CHEMBL511000;44563281;CHEMBL51100...,CC1(C)CCC2(C(=O)Nc3ccc(C(=O)O)cc3)CCC3(C)C(=CC...,AAGZFGSRAZMRCS,AAGZFGSRAZMRCS-UHFFFAOYSA-N,InChI=1S/C37H53NO4/c1-32(2)18-20-37(31(42)38-2...,"""AuxInfo=1/1/N:1,3,34,35,28,22,40,12,17,11,18,...",P18031_WT,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.945723,55.179680
2,AATACZKOPAGNPL_on_P18031_WT,High,ChEMBL30,CHEMBL4160013,CCOc1ccc(OC(=O)CSc2nnc(-c3cc(O)c(O)cc3)o2)cc1,AATACZKOPAGNPL,AATACZKOPAGNPL-UHFFFAOYSA-N,InChI=1S/C18H16N2O6S/c1-2-24-12-4-6-13(7-5-12)...,"""AuxInfo=1/0/N:1,2,24,5,27,6,26,23,18,11,17,4,...",P18031_WT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.630247,-24.217190
3,AAWBMDOJDQKGGQ_on_P18031_WT,High,ChEMBL30,CHEMBL3759405,CCCOc1ccc(-c2cccc(-c3noc(Cc4c[nH]c5ccccc45)n3)...,AAWBMDOJDQKGGQ,AAWBMDOJDQKGGQ-UHFFFAOYSA-N,InChI=1S/C25H22N4O2/c1-2-14-30-19-12-10-17(11-...,"""AuxInfo=1/0/N:1,2,25,24,11,26,23,10,12,7,30,6...",P18031_WT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.455896,-43.069510
4,ABDQNRCWGSGNBQ_on_P18031_WT,High,ChEMBL30,CHEMBL197929,O=C(O)COc1ccc(S(=O)(=O)N(Cc2ccc(C(F)(F)P(=O)(O...,ABDQNRCWGSGNBQ,ABDQNRCWGSGNBQ-UHFFFAOYSA-N,"InChI=1S/C21H20F2NO8PS2/c22-21(23,33(27,28)29)...","""AuxInfo=1/1/N:31,30,16,27,17,26,7,35,8,34,32,...",P18031_WT,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.058834,5.535724


## Chemical Space Visualization

Now, we can do our first data analysis of this data set, which will be a visualization of the chemical space covered by the compounds tested against the A2A receptor in public literature. In order to do that, we will use another feature of `scaffviz`, which is interactive plotting with the [`molplotly`](https://github.com/wjm41/molplotly) package. We can configure the visualization in many different ways, but for now we just want to use the Morgan fingeprints we calculated to calculate a t-SNE embedding to place similar compounds in the same parts of the plot. We also color the points based on the pChEMBL median value extracted from the Papyrus data set. The higher this value, the more affinity/effect the compound should have on the A2A receptor. Generally, hits with pChEMBL of 6.5 and higher are considered as having significant biological effect:

In [12]:
from scaffviz.depiction.plot import Plot
from scaffviz.clustering.manifold import TSNE

# make an interactive plot that will use t-SNE to embed the data set in 2D (all current descriptors in the data set will be used)
plt = Plot(dataset, TSNE())

# start the server, you can open the plot in the browser
plt.plot(
    recalculate=False, 
    color_by="pchembl_value_Median",
    color_style="continuous",
    color_continuous_scale="rdylgn",
    card_data=["pchembl_value_Median", "all_doc_ids", "source"],
    title_data="Activity_ID",
    viewport_height=800,
    port=8888
)

<IPython.core.display.Javascript object>

This plot you are seeing is running as a seperate web application embedded into this one. So if you are running this notebook on [localhost:8888](http://localhost:8888), you can navigate to [localhost:9090](http://localhost:9090) to see te full depiction on its own page. You should be able to see clusters of similar molecules and regions of chemical space with active/inactive molecules should be discernible. Therefore, such visualization could be useful to assess how many molecules of what activity we have in the data set and get an idea how similar they are to each other. It is also possible to identify potential activity cliffs (regions where very similar molecules have vastly different activity). 

In the [next part of the tutorial](./qsar.ipynb), we will show a simple worfklow where we use this data to train a simple machine learning model that can learn how to predict whether a compound is likely to be active or inactive on A2A receptor.